In [0]:
import numpy as np
from keras.models import Model, Sequential
from keras.layers import Dense, GRU, Activation,Input, Reshape, Embedding, Flatten, LeakyReLU, Conv2D, MaxPool2D
import random
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
import scipy.sparse as sp
import pandas as pd

In [0]:
dataset = pd.read_csv("https://raw.githubusercontent.com/datasets/tic-tac-toe/master/data/tic-tac-toe.csv")
dataset = dataset.loc[dataset['class']==True]
dataset = dataset.iloc[:,:-1]
dataset = np.array(dataset)

In [0]:
np.shape(dataset)

In [0]:
def gen_data(gameboard):
  x_data = []
  y_data = []

  while 'x' in gameboard:
    x = np.argwhere(gameboard=='x')
    x = np.reshape(x,x.size)
    o = np.argwhere(gameboard=='o')
    o = np.reshape(o,o.size)
    if x.size-o.size != 1:
      print(str(x.size)+" "+str(o.size))
    
    index = random.sample(set(x),1)
    gb = np.zeros(9)
    gb[index] = 1
    y_data = np.append(y_data,gb)
    gameboard[index] = 'b'
    x_data = np.append(x_data,gameboard)
    if 'o' not in gameboard:
      break
    index = random.sample(set(o),1)        
    gameboard[index] = 'b'
  x_data = np.reshape(x_data,(-1,9))
  y_data = np.reshape(y_data,(-1,9))
  return x_data,y_data

In [0]:
x_dataset = []
y_dataset = []
for data in dataset:
  x,y = gen_data(data)
  for i,j in zip(x,y):
    if i not in x_dataset or j not in y_dataset:
      x_dataset = np.append(x_dataset,i)
      y_dataset = np.append(y_dataset,j)
x_dataset = np.reshape(x_dataset,(-1,9))
y_dataset = np.reshape(y_dataset,(-1,9))  

In [0]:
OE = OrdinalEncoder(categories='auto')
SS = StandardScaler()
def gameboard_transform(gameboard,OE=OE,SS=SS):
  gameboard = OE.fit_transform(gameboard)
  gameboard = SS.fit_transform(gameboard)
  return gameboard
def inverse_gameboard_transform(gameboard,OE=OE,SS=SS):
  gameboard = SS.inverse_transform(gameboard)
  gameboard = OE.inverse_transform(gameboard)
  return gameboard

In [0]:
random.sample(list(x_dataset),5)

In [0]:
x_dataset = gameboard_transform(x_dataset)

In [0]:
print(np.shape(x_dataset))
print(np.shape(y_dataset))

In [0]:
model = Sequential()
#model.add(Reshape((1,3,3),input_shape=(9,)))
#model.add(Conv2D(9,kernel_size=(3,3),padding='same',activation='sigmoid'))
#model.add(MaxPool2D(pool_size=(2,2),strides=(1,1),padding='same'))
#model.add(Flatten())
model.add(Dense(9**3,activation='tanh',input_shape=(9,)))
model.add(Dense(9*2,activation='tanh'))
model.add(Dense(9,activation='tanh'))
model.add(Dense(9,activation='softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

model.summary()

In [0]:
model.fit(x=x_dataset,y=y_dataset,epochs=100,batch_size=32)

In [0]:
model.save('tictactoe.h5')

In [0]:
def is_game_over(game_board):
  game_board = np.reshape(game_board,9)
  win_seq = [[0,1,2],[3,4,5],[6,7,8],[0,3,6],[1,4,7],[2,5,8],[0,4,8],[2,4,6]]
  for i in win_seq:
   if game_board[i[0]] == game_board[i[1]] == game_board[i[2]] == 'x':
     return 'X'
   if game_board[i[0]] == game_board[i[1]] == game_board[i[2]] == 'o':
     return 'O'
  if 'b' not in game_board:
   return 'D'
  return 'N'      

In [0]:
x = random.sample(list(x_dataset),1)
print(x)
y = model.predict(np.reshape(x,(1,9)))
x = inverse_gameboard_transform(x)
print(np.argmax(y))

In [0]:
gameboard = [['b','b','b','b','b','b','b','b','b']]

def print_gameboard(x):
  print(x[0][0]+" "+x[0][1]+" "+x[0][2])
  print(x[0][3]+" "+x[0][4]+" "+x[0][5])
  print(x[0][6]+" "+x[0][7]+" "+x[0][8])
  

while is_game_over(gameboard) == 'N':
  x = OE.transform(gameboard)
  x = SS.transform(x)
  m_x = model.predict(x)
  index = np.argmax(m_x)
  gameboard[0][index] = 'x'
  print_gameboard(gameboard)
  if is_game_over(gameboard) == 'X':
    print("Computer wins")
    break
  if is_game_over(gameboard) == 'D':
    print("Draw")
    break
  o = int(input("Enter between 1-9 : "))
  o = o-1
  while gameboard[0][o] == 'x':
    o = int(input("Enter between 1-9 : "))
    o = o-1
  gameboard[0][o] = 'o'
  if is_game_over(gameboard) == 'O':
    print("You win")
    break
  if is_game_over(gameboard) == 'D':
    print("Draw")
    break